In [4]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# doc du lieu tu file
data = pd.read_excel('flight.xlsx')

# trich xuat ma san bay
def extract_airport_code(airport_name):
    match = re.search(r'\((.*?)\)', airport_name)
    if match:
        return match.group(1)  # Trả về mã sân bay
    return None

data['airport_code_from'] = data['airport_from'].apply(extract_airport_code)
data['airport_code_to'] = data['airport_to'].apply(extract_airport_code)

# loc du lieu chi vietjet
vietjet_data = data[data['code_name'] == 'Vietjet']

# lay cac cap san bay (airport_code_from, airport_code_to)
unique_airports = vietjet_data[['airport_code_from', 'airport_code_to']].drop_duplicates()

# Dictionary luu tru khoang cach da tinh
airport_distances = {}

# tra cuu toa do tu ma san bay
def get_airport_coordinates(airport_code):
    geolocator = Nominatim(user_agent="airport_distance_calculator")
    location = geolocator.geocode(f"{airport_code} airport")
    if location:
        return (location.latitude, location.longitude)
    return None

# tinh khoang cach va luu vao dictionary
def get_airport_distance(airport_from, airport_to):
    if (airport_from, airport_to) in airport_distances:
        return airport_distances[(airport_from, airport_to)]
    elif (airport_to, airport_from) in airport_distances:
        return airport_distances[(airport_to, airport_from)]
    else:
        coords_from = get_airport_coordinates(airport_from)
        coords_to = get_airport_coordinates(airport_to)
        
        if coords_from and coords_to:
            distance = geodesic(coords_from, coords_to).kilometers
            airport_distances[(airport_from, airport_to)] = distance
            airport_distances[(airport_to, airport_from)] = distance  # Khoảng cách đối xứng
            return distance
        else:
            return None

# tinh khoang cach san bay duy nhat
for _, row in unique_airports.iterrows():
    airport_from = row['airport_code_from']
    airport_to = row['airport_code_to']
    distance = get_airport_distance(airport_from, airport_to)
    if distance is not None:
        # gan vao dictionary
        airport_distances[(airport_from, airport_to)] = distance
        airport_distances[(airport_to, airport_from)] = distance

# tinh distance moi chuyen
def calculate_distance(row):
    airport_from = row['airport_code_from']
    airport_to = row['airport_code_to']
    distance = airport_distances.get((airport_from, airport_to))
    return distance

vietjet_data = data[data['code_name'] == 'Vietjet'].copy()

vietjet_data['distance'] = vietjet_data.apply(calculate_distance, axis=1)

vietjet_data_cleaned = vietjet_data.dropna(subset=['f_price', 'fees'])
vietjet_data_cleaned['f_time_from'] = pd.to_datetime(vietjet_data_cleaned['f_time_from'], format='%H:%M:%S %d/%m/%Y')
vietjet_data_cleaned['f_time_to'] = pd.to_datetime(vietjet_data_cleaned['f_time_to'], format='%H:%M:%S %d/%m/%Y')
vietjet_data_cleaned['ticket_type'] = vietjet_data_cleaned['type'].apply(lambda x: 0 if x == 'Eco' else 1)

vietjet_data_cleaned.to_excel('vietjet_flight_data.xlsx', index=False)

# kiểm tra
print(vietjet_data_cleaned.head())

              id code code_name f_code         f_time_from  \
0  vj16169846100   vj   Vietjet  VJ122 2021-04-01 04:50:00   
2  vj16169846231   vj   Vietjet  VJ176 2021-04-01 05:25:00   
4  vj16169846302   vj   Vietjet  VJ120 2021-04-01 06:00:00   
6  vj16169846383   vj   Vietjet  VJ132 2021-04-01 06:05:00   
8  vj16169846454   vj   Vietjet  VJ134 2021-04-01 07:00:00   

            f_time_to  f_price    fees  total_price    from  \
0 2021-04-01 07:10:00   199000  650900       849900  TP HCM   
2 2021-04-01 07:35:00   199000  650900       849900  TP HCM   
4 2021-04-01 08:10:00   199000  650900       849900  TP HCM   
6 2021-04-01 08:05:00   199000  650900       849900  TP HCM   
8 2021-04-01 09:10:00   409000  661900      1070900  TP HCM   

         airport_from      to     airport_to type airport_code_from  \
0  Tân Sơn Nhất (SGN)  Hà Nội  Nội Bài (HAN)  Eco               SGN   
2  Tân Sơn Nhất (SGN)  Hà Nội  Nội Bài (HAN)  Eco               SGN   
4  Tân Sơn Nhất (SGN)  Hà Nội  Nội 